# Clustering


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sns
from pyclustertend import hopkins, vat

#### _1.1 Haga el preprocesamiento del dataset, explique qué variables no aportan información a la generación de grupos y por qué. Describa con qué variables calculará los grupos_


<small>

**Variables que no aportan información para clústering:**

Es importante resaltar que clustering se maneja por medio de valores numéricos. Por lo tanto, los valores que no aportan información para clústering son:

1. **id:** Es solo el identificador único, no tiene relevancia para la agrupación.
2. **homepage:** Debido a que es una URL, no aporta valor numérico o categórico relevante.
3. **originalTile y title:** Es el nombre de las películas, no aportan información cuantificable.
4. **originalLanguage:** Es una variable categórica con demasiadas clases, no es relevante para el clústering.
5. **releaseDate:** Es una fecha que no aporta información numérica útil directamente.
6. **director, actor, actorsCharacter:** Son nombres de personas, no datos útiles para análisis cuantitativo.
7. **video:** Contiene valores _True/False_, lo cual no es tan útil para el clustering en este caso.
8. **productionCompany, productionCompanyCountry, productionCountry, genres:** Su formato textual dificulta el uso de clustering.

</small>


<small>

**Variables Seleccionadas para Calcular los Grupos**

Cabe resaltar que todas las variables seleccionadas para calcular grupos, son de tipo cuantitativo.

1. budget
2. revenue
3. popularity
4. voteAvg
5. voteCount
6. runtime
7. genresAmount
8. productionCoAmount
9. productionCountriesAmount
10. actosPopularity
11. actorsAmount
12. castWomenAmount
13. castMenAmount
    </small>


#### _1.2 Analice la tendencia al agrupamiento usando el estadístico de Hopkings y la VAT (Visual Assessment of cluster Tendency). Esta última hágala si es posible, teniendo en cuenta las dimensiones del conjunto de datos. Discuta sus resultados e impresiones._


In [8]:
file_path = "movies.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")

# Preprocesar 'actorsPopularity' para calcular el promedio de popularidades
def calculate_mean_popularity(popularity_string):
    try:
        popularity_values = list(map(float, popularity_string.split('|')))
        if len(popularity_values) > 0:
            return np.mean(popularity_values)
        else:
            return 0
    except:
        return 0

df['actorsPopularity'] = df['actorsPopularity'].apply(calculate_mean_popularity)

# Variables seleccionadas para clustering
features = [
    "budget", "revenue", "runtime", "actorsPopularity",
    "popularity", "voteAvg", "voteCount", "genresAmount",
    "productionCoAmount", "productionCountriesAmount",
    "actorsAmount", "castWomenAmount", "castMenAmount"
]

# Filtrar solo las columnas relevantes
df_cluster = df[features].copy()

# Convertir castWomenAmount y castMenAmount a numérico (por si tienen strings)
df_cluster["castWomenAmount"] = pd.to_numeric(df_cluster["castWomenAmount"], errors="coerce")
df_cluster["castMenAmount"] = pd.to_numeric(df_cluster["castMenAmount"], errors="coerce")

# Manejo de valores nulos con la mediana
df_cluster[df_cluster.select_dtypes(include=[np.number]).columns] = df_cluster.select_dtypes(include=[np.number]).fillna(df_cluster.median(numeric_only=True))

# Normalizar los datos para clustering
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Calcular el estadístico de Hopkins usando pyclustertend
hopkins_score = hopkins(df_scaled, df_scaled.shape[0])
print(f"Estadístico de Hopkins: {hopkins_score:.4f}")

# Visual Assessment of Cluster Tendency (VAT) usando pyclustertend
if df_scaled.shape[0] <= 5000:  # Verificar el tamaño para evitar problemas de visualización
    vat(df_scaled)
else:
    print("VAT omitida por el tamaño de los datos.")

Estadístico de Hopkins: 0.0177
VAT omitida por el tamaño de los datos.


<small>

### **Análisis de Hopkins**

El resultado obtenido del estadístico de Hopkins fue de **0.0177**, lo cual es significativamente bajo y sugiere una alta propensión al agrupamiento dentro del conjunto de datos analizado. En el contexto de la minería de datos, un valor de Hopkins tan bajo es indicativo de que los datos no están distribuidos de manera aleatoria y, por lo tanto, existen patrones intrínsecos que pueden ser identificados mediante técnicas de clustering.

### **Análisis de VAT**

La omisión de la Visual Assessment of Cluster Tendency (VAT) se debió al tamaño considerable del dataset, que plantea desafíos computacionales y de visualización. La técnica VAT es una herramienta útil para evaluar visualmente si un conjunto de datos contiene grupos naturales discernibles, pero su eficacia disminuye con el aumento del tamaño del dataset debido a la complejidad de la representación visual de las distancias entre datos en una matriz de disimilitud.
</small>
